# Install Dependencies
Headless rendering dependencies



In [0]:
# Remove " > /dev/null" to see what is going on under the hood
!apt-get update > /dev/null
!pip install pyvirtualdisplay > /dev/null
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null

Mujoco-py, dependencies, and license

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
!apt-get update -q > /dev/null\
    && DEBIAN_FRONTEND=noninteractive apt-get install -y \
    curl \
    git \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common \
    net-tools \
    unzip \
    vim \
    virtualenv \
    wget \
    xpra \
    xserver-xorg-dev > /dev/null \
    && apt-get clean > /dev/null \
    && rm -rf /var/lib/apt/lists/* > /dev/null

#!DEBIAN_FRONTEND=noninteractive add-apt-repository --yes ppa:deadsnakes/ppa && apt-get update > /dev/null
#!DEBIAN_FRONTEND=noninteractive apt-get install --yes python3.6-dev python3.6 python3-pip > /dev/null

!curl -o /usr/local/bin/patchelf https://s3-us-west-2.amazonaws.com/openai-sci-artifacts/manual-builds/patchelf_0.9_amd64.elf > /dev/null \
    && chmod +x /usr/local/bin/patchelf > /dev/null

!export LANG=C.UTF-8 > /dev/null
!rm -rf /root/.mujoco > /dev/null
!mkdir -p /root/.mujoco > /dev/null \
    && wget https://www.roboti.us/download/mjpro150_linux.zip -O mujoco.zip > /dev/null \
    && unzip mujoco.zip -d /root/.mujoco > /dev/null \
    && rm mujoco.zip > /dev/null
!cp ./mjkey.txt /root/.mujoco/ > /dev/null
!export LD_LIBRARY_PATH=/root/.mujoco/mjpro150/bin:${LD_LIBRARY_PATH} > /dev/null
!rm -rf /mujoco_py > /dev/null && mkdir /mujoco_py > /dev/null
# Copy over just requirements.txt at first. That way, the Docker cache doesn't
# expire until we actually change the requirements.
!git clone https://github.com/openai/mujoco-py.git && cd mujoco-py \
&& cp -r ./* /mujoco_py/ && cd /mujoco_py && pip install -r requirements.txt > /dev/null \
&& pip install -r requirements.dev.txt && pytest > /dev/null && python setup.py install > /dev/null

In [0]:
!pip install -U mujoco-py

OpenAI Gym & Dependencies

In [0]:
#!apt-get update > /dev/null
#!apt-get install -y python-pyglet python3-opengl zlib1g-dev libjpeg-dev patchelf \
#        cmake swig libboost-all-dev libsdl2-dev libosmesa6-dev xvfb ffmpeg > /dev/null
#!pip install gym[all] > /dev/null

Baselines Dependencies

In [0]:
#!apt-get update > /dev/null
#!apt-get install cmake libopenmpi-dev python3-dev zlib1g-dev  > /dev/null

# Imports and Helper functions


In [0]:
import gym
import mujoco_py
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

import os
os.environ['LD_LIBRARY_PATH'] += ":/root/.mujoco/mjpro150/bin"
os.environ['LD_LIBRARY_PATH'] += ":/usr/lib/nvidia-396"

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Mujoco Simulation

In [0]:
import mujoco_py
import os
mj_path, _ = mujoco_py.utils.discover_mujoco()
xml_path = os.path.join(mj_path, 'model', 'humanoid.xml')
model = mujoco_py.load_model_from_path(xml_path)
sim = mujoco_py.MjSim(model)

print(sim.data.qpos)
sim.step()
print(sim.data.qpos)

In [0]:
observation = env.reset()
while True:
    env.render()
    action = env.action_space.sample()      
    observation, reward, done, info = env.step(action) 
    if done: 
      break; 
env.close()
show_video()